<a href="https://colab.research.google.com/github/attiaamaly/testrepo/blob/main/FINAL_PYTHON_CODE_FOR_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr

# =========================
# OFFICIAL DATA (IPCC/NREL/OWID)
# =========================
EMISSION_FACTORS = pd.DataFrame({
    "source": ["solar","wind","hydro","nuclear","biomass","gas","oil","coal"],
    "g_per_kwh": [45, 12, 24, 12, 230, 490, 650, 820]
})

# =========================
# HELPERS
# =========================
def normalize_mix(mix):
    tot = sum(mix.values())
    return {k: (0 if tot == 0 else 100*v/tot) for k, v in mix.items()}

def compute_emissions(mix_dict, factors_df):
    mix = normalize_mix(mix_dict)
    df = pd.DataFrame(list(mix.items()), columns=["source","share_pct"]).merge(
        factors_df, on="source", how="left"
    )
    df["weighted_g"] = df["share_pct"] * df["g_per_kwh"] / 100.0
    intensity = df["weighted_g"].sum()  # g/kWh
    df = df[["source","share_pct","g_per_kwh","weighted_g"]].sort_values("weighted_g", ascending=False)
    df.columns = ["Source","Share (%)","CO₂ factor (g/kWh)","Weighted (g/kWh)"]
    return intensity, df

def color_band(intensity_g_per_kwh):
    if intensity_g_per_kwh <= 150: return "Green","🟢"
    if intensity_g_per_kwh <= 350: return "Orange","🟠"
    return "Red","🔴"

def ets_cost(intensity_g_per_kwh, annual_mwh, carbon_price_eur_per_t):
    kwh = annual_mwh * 1000.0
    tco2 = (intensity_g_per_kwh / 1_000_000.0) * kwh
    cost = tco2 * carbon_price_eur_per_t
    return round(tco2, 2), round(cost, 0)

def traffic_light_plot(intensity):
    fig, ax = plt.subplots(figsize=(5, 1.2))
    ax.barh([0], [150], color="#2ecc71")                 # Green 0-150
    ax.barh([0], [200], left=[150], color="#f39c12")     # Orange 150-350
    ax.barh([0], [650], left=[350], color="#e74c3c")     # Red 350-1000
    ax.plot([intensity], [0], marker="v", color="black", markersize=10)
    ax.set_xlim(0, 1000); ax.set_ylim(-0.5, 0.5)
    ax.set_yticks([]); ax.set_xlabel("gCO₂/kWh")
    ax.set_title(f"CO₂ intensity = {intensity:.1f} g/kWh")
    plt.tight_layout()
    return fig

# =========================
# SMART RECOMMENDATIONS
# =========================
SERVICES = {
    "audit": {"title":"Energy Performance Audit","desc":"Baseline your sites, detect inefficiencies, quantify savings & CO₂ abatement.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "iso50001": {"title":"ISO 50001 Monitoring & Reporting","desc":"Continuous metering, dashboards and compliance reporting.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "ems": {"title":"Digital EMS & Demand Response","desc":"Automate peak shaving and shift loads to lower-cost, low-carbon hours.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "pv_epc": {"title":"Solar PV Feasibility & EPC","desc":"Site study, design, procurement and construction for on-site PV.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "wind_assess": {"title":"Onshore Wind Assessment & Grid Study","desc":"Resource assessment, permitting, and interconnection planning.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "battery": {"title":"Battery Storage Sizing & Integration","desc":"Stabilize variable renewables and cut peak charges/curtailment.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "retrofit": {"title":"Retrofit & Modernization","desc":"Upgrade turbines/boilers, drives and controls to raise efficiency.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "predictive": {"title":"Predictive Maintenance & O&M","desc":"SCADA analytics, condition monitoring and service contracts.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "h2_ready": {"title":"H₂-Ready Gas Turbine Roadmap","desc":"Blend green hydrogen and plan the fuel switch safely.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "fuel_switch": {"title":"Fuel Switch Study (Coal/Oil → Gas/H₂)","desc":"Techno-economic pathway to lower-carbon fuels and emissions.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "grid_stability": {"title":"Grid Stability & Power Quality","desc":"SVC/STATCOM, synchronous condensers and protection upgrades.","url":"https://www.siemens-energy.com/global/en/home.html"},
    "performance_dash": {"title":"Performance Analytics Dashboard","desc":"Track KPIs, alarms and CO₂ intensity in real time.","url":"https://www.siemens-energy.com/global/en/home.html"}
}

def get_mix_signals(mix_norm):
    renew = mix_norm.get("solar",0)+mix_norm.get("wind",0)+mix_norm.get("hydro",0)+mix_norm.get("biomass",0)
    fossil = mix_norm.get("gas",0)+mix_norm.get("oil",0)+mix_norm.get("coal",0)
    gas, oil, coal = mix_norm.get("gas",0), mix_norm.get("oil",0), mix_norm.get("coal",0)
    solar, wind = mix_norm.get("solar",0), mix_norm.get("wind",0)
    nuclear, hydro, biomass = mix_norm.get("nuclear",0), mix_norm.get("hydro",0), mix_norm.get("biomass",0)
    intermittents = solar + wind
    baseload = nuclear + hydro + biomass + gas
    dominant = max(mix_norm, key=mix_norm.get)
    return {"renew":renew,"fossil":fossil,"gas":gas,"oil":oil,"coal":coal,
            "solar":solar,"wind":wind,"nuclear":nuclear,"hydro":hydro,"biomass":biomass,
            "intermittents":intermittents,"baseload":baseload,"dominant":dominant}

def recommend_services(mix_norm, band):
    s = get_mix_signals(mix_norm)
    rec_keys = []

    # Always useful visibility
    rec_keys += ["audit", "iso50001", "ems"]

    # Fossil-heavy profiles
    if s["fossil"] >= 50:
        rec_keys += ["retrofit", "predictive"]
        if s["coal"] >= 15 or s["oil"] >= 10:
            rec_keys += ["fuel_switch", "grid_stability"]
        if s["gas"] >= 25:
            rec_keys += ["h2_ready"]

    # Low renewables or untapped potential
    if s["renew"] < 40:
        rec_keys += ["pv_epc", "wind_assess", "battery"]

    # High intermittency → storage & stability
    if s["intermittents"] >= 45:
        rec_keys += ["battery", "grid_stability"]

    # Baseload present but not fully optimized
    if s["baseload"] >= 50 and s["renew"] < 60 and s["gas"] <= 30:
        rec_keys += ["retrofit", "performance_dash"]

    # Band-specific nudges
    if band == "Red":
        rec_keys += ["fuel_switch", "retrofit", "battery"]
    elif band == "Orange":
        rec_keys += ["retrofit", "performance_dash"]
    else:  # Green
        rec_keys += ["performance_dash"]

    # De-duplicate & cap
    seen, out = set(), []
    for k in rec_keys:
        if k in SERVICES and k not in seen:
            out.append(SERVICES[k]); seen.add(k)
    return out[:6]

def services_md(services):
    return "\n".join([f"- **[{s['title']}]({s['url']})** — {s['desc']}" for s in services])

# =========================
# PIPELINE
# =========================
def run_pipeline(annual_mwh, carbon_price,
                 solar, wind, hydro, nuclear, biomass, gas, oil, coal):

    mix = {"solar":solar, "wind":wind, "hydro":hydro, "nuclear":nuclear,
           "biomass":biomass, "gas":gas, "oil":oil, "coal":coal}
    mix_norm = normalize_mix(mix)

    intensity, table = compute_emissions(mix, EMISSION_FACTORS)
    band, emoji = color_band(intensity)

    fig = traffic_light_plot(intensity)

    # ETS cost only for Orange/Red (you wanted Red; you can switch to band == "Red")
    message = f"**Band:** {emoji} **{band}**"
    if band == "Red":
        tco2, cost = ets_cost(intensity, annual_mwh, carbon_price)
        message += (f"\n\n**EU ETS cost estimate:** ~€{cost:,.0f}/year "
                    f"(emissions ≈ {tco2:,.2f} tCO₂; "
                    f"assumptions: {annual_mwh:,} MWh/yr, €{carbon_price}/tCO₂)")
    elif band == "Orange":
        message += "\n\nOpportunity to reduce emissions and energy cost with targeted actions."

    services = recommend_services(mix_norm, band)
    rec_md = services_md(services)

    footer = ("\n\n_Data source: IPCC/NREL medians via Our World in Data (gCO₂e/kWh). "
              "Thresholds for demo: Green ≤150, Orange 150–350, Red ≥350._")

    summary_md = (f"**CO₂ intensity:** **{intensity:.1f} g/kWh**  \n"
                  f"{message}  \n\n**Recommended Siemens services:**\n{rec_md}{footer}")

    return summary_md, table, fig

# =========================
# UI
# =========================
with gr.Blocks(title="Siemens Energy – CO₂ Impact & Advisory Dashboard") as demo:
    gr.Markdown("## Siemens Energy – CO₂ Impact & Advisory Dashboard")
    gr.Markdown("Choose your **energy mix**. See the **CO₂ table** (official factors), "
                "a **Green/Orange/Red** chart, an **EU ETS cost** if in Red, "
                "and **Siemens service recommendations**.")

    with gr.Row():
        with gr.Column():
            annual_mwh   = gr.Slider(100, 500000, value=10000, step=100, label="Annual electricity (MWh)")
            carbon_price = gr.Slider(30, 150, value=75, step=1, label="Carbon price (€/tCO₂)")
        with gr.Column():
            solar   = gr.Slider(0, 100, 30, label="Solar (%)")
            wind    = gr.Slider(0, 100, 25, label="Wind (%)")
            hydro   = gr.Slider(0, 100, 10, label="Hydro (%)")
            nuclear = gr.Slider(0, 100, 10, label="Nuclear (%)")
            biomass = gr.Slider(0, 100, 5,  label="Biomass (%)")
            gas     = gr.Slider(0, 100, 15, label="Gas (%)")
            oil     = gr.Slider(0, 100, 2,  label="Oil (%)")
            coal    = gr.Slider(0, 100, 3, label="Coal (%)")

    btn = gr.Button("Compute")
    summary = gr.Markdown()
    table   = gr.Dataframe(wrap=True)
    chart   = gr.Plot()

    btn.click(
        run_pipeline,
        [annual_mwh, carbon_price, solar, wind, hydro, nuclear, biomass, gas, oil, coal],
        [summary, table, chart]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://067e5c109d641f1844.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
